<a href="https://colab.research.google.com/github/GeorgeGPUGH/Machine-Learning/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Data Preprocessing
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Convert labels to numerical format
df_train['label'] = df_train['label'].map({'ham': 0, 'spam': 1})
df_test['label'] = df_test['label'].map({'ham': 0, 'spam': 1})

# Tokenize and pad messages
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df_train['message'])

In [ ]:
# Split into train and test data
X_train = tokenizer.texts_to_sequences(df_train['message'])
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=50)

X_test = tokenizer.texts_to_sequences(df_test['message'])
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=50)

y_train = df_train['label'].values
y_test = df_test['label'].values

In [ ]:
# Model Building
model = keras.Sequential([
    keras.layers.Embedding(len(tokenizer.word_index) + 1, 16, input_shape=(50,)),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Model Training
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_tokenized = tokenizer.texts_to_sequences([pred_text])
  pred_padded = keras.preprocessing.sequence.pad_sequences(pred_tokenized, maxlen=50)
  prediction = model.predict(pred_padded)[0][0]
  return [prediction, 'ham' if prediction < 0.5 else 'spam']

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
